In [8]:
# === 하위 폴더 모든 CSV에서 '세부 품목명' 모아 카운트 (저장 없이 화면 표시) ===
import os, glob, warnings
from collections import Counter, defaultdict
import pandas as pd

warnings.filterwarnings("ignore")  # 워닝 숨김
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)

# 세부 품목/품목 컬럼 후보
SUBITEM_CANDIDATES = ["세부 품목명", "세부 품목", "세부품목","품목명"]
ITEM_CANDIDATES    = ["품목명", "품목"]
ENCODINGS = ["utf-8-sig", "utf-8", "cp949", "euc-kr"]

def _read_csv_robust(path):
    last_err = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f"CSV 로딩 실패: {path} / {last_err}")

def _find_first_present(cols, candidates):
    cols_norm = [str(c).strip() for c in cols]
    for cand in candidates:
        if cand in cols_norm:
            return cand
    return None

def extract_subitem_counts_nosave(root_dir: str):
    """
    root_dir 하위 모든 *.csv에서 세부 품목명을 모아 카운트하고 DataFrame으로 반환.
    - 파일 저장 없이 화면 표시 용.
    - '세부 품목명/세부 품목/세부품목' 중 존재하는 첫 컬럼 사용.
    """
    pattern = os.path.join(root_dir, "**", "*.csv")
    paths = sorted(glob.glob(pattern, recursive=True))
    if not paths:
        raise SystemExit(f"[중지] CSV를 찾지 못했습니다: {root_dir}")

    total_counter = Counter()
    appears_in_files = defaultdict(set)  # 값 -> {파일}
    per_file_map = {}                    # 파일 -> [유니크값]

    for fp in paths:
        try:
            df = _read_csv_robust(fp)
        except Exception:
            # 문제 파일은 조용히 스킵
            continue

        df.columns = [str(c).strip() for c in df.columns]
        sub_col = _find_first_present(df.columns, SUBITEM_CANDIDATES)
        if sub_col is None:
            continue

        s = (
            df[sub_col]
              .astype(str)
              .str.strip()
              .replace({"": pd.NA, "nan": pd.NA, "None": pd.NA})
              .dropna()
        )
        if s.empty:
            continue

        vals = s.unique().tolist()
        per_file_map[fp] = vals

        total_counter.update(s.tolist())
        for v in vals:
            appears_in_files[v].add(fp)

    rows = []
    for val, cnt in total_counter.most_common():
        files = sorted(list(appears_in_files[val]))
        rows.append({
            "세부품목명": val,
            "count": cnt,
            "n_files": len(files),
            "files": " | ".join(files),
        })

    return pd.DataFrame(rows), per_file_map

# ===== 실행 =====
ROOT = r"C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop"
result_df, per_file = extract_subitem_counts_nosave(ROOT)

# 화면에 바로 보기
from IPython.display import display
print(f"[INFO] 세부품목명 개수: {len(result_df)}  |  스캔한 CSV 수: "
      f"{len(glob.glob(os.path.join(ROOT, '**', '*.csv'), recursive=True))}")
display(result_df.head(50))   # 상위 50개 미리보기

# (선택) 세부품목명별로 등장 파일수/총건수만 보고 싶다면:
# display(result_df[["세부품목명", "count", "n_files"]].head(50))

# (선택) 파일별 유니크 세부품목 미리보기
# for fp, vals in list(per_file.items())[:3]:
#     print(f"\n[파일] {fp}\n[유니크 세부품목(일부)] {vals[:10]}")


[INFO] 세부품목명 개수: 52  |  스캔한 CSV 수: 36


,세부품목명,count,n_files,files
0,녹광,7308,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\_merged\청고추_merged.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\청고추_drop.csv
1,오이맛고추,7308,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\_merged\청고추_merged.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\청고추_drop.csv
2,청양,7308,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\_merged\청고추_merged.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\청고추_drop.csv
3,청초(일반),7308,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\_merged\청고추_merged.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\청고추_drop.csv
4,노랑파프리카,7308,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\_merged\파프리카_merged.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\파프리카_1_drop.csv
5,빨강파프리카,7308,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\_merged\파프리카_merged.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\파프리카_1_drop.csv
6,마늘,6102,2,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\마늘(통합)_최종_데이터.csv | C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\마늘_drop.csv
7,수미,3654,1,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\감자_drop.csv
8,밤고구마,3654,1,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\고구마_drop.csv
9,호박고구마,3654,1,C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw\drop\고구마_drop.csv
